# Session 1.1: Python Object Identity, Interning, and Headers
This notebook dives deep into how Python manages object identity, memory, and performance-critical behaviors.

## 1. `id()` vs `is` vs `==`
In Python:
- `==` checks for value equality.
- `is` checks for object identity (i.e., same memory location).
- `id()` returns the unique identity (memory address) of an object.

Let's demonstrate with simple examples:

In [2]:
a = 256
b = 256
c = 1000
d = 1000
print("a is b:", a is b)       # True because of integer interning
print("id(a):", id(a), "id(b):", id(b))
print("c is d:", c is d)       # False because of different objects
print("id(c):", id(c), "id(d):", id(d))
b = 200
print("a is b:", a is b)
x = [1, 2]
y = [1, 2]
print("x == y:", x == y)       # True: value equality
print("x is y:", x is y)       # False: different objects
print("id(x):", id(x), "id(y):", id(y))
# Lets see if a an  integer defined previosuly immutable
print("id if a is", id(a))
# Let change the value of a to 255 and see what happens
a = 255
print("id if a is", id(a))

a is b: True
id(a): 10759016 id(b): 10759016
c is d: False
id(c): 133823873100848 id(d): 133823873101072
a is b: False
x == y: True
x is y: False
id(x): 133823873426624 id(y): 133823873418304
id if a is 10759016
id if a is 10758984


## 2. Interning of Small Integers and Strings
Python automatically interns:
- Small integers (typically from -5 to 256)
- Some short strings used as identifiers

Manual interning can be done using `sys.intern()` to save memory or speed up dictionary lookups.

In [ ]:
# prompt: Lets see the rationale of python making objects like  an int variable immutable with an example

# The output shows that the id of 'a' changes when you reassign it a different value.
# This demonstrates that integers in Python are immutable.  When you do `a = 255`,
# you are not modifying the original integer object representing 256. Instead, you are
# creating a *new* integer object with the value 255 and assigning the variable 'a'
# to point to this new object.  The old object (256) remains unchanged.

# In other words:  you cannot change the value of an integer object itself.  You
# can only reassign the variable to point to a *different* integer object with
# a new value.  This is the core principle of immutability in Python.
a = 256
print("id if a is", id(a))
a = 255 #  This creates a NEW integer object.  'a' now points to this new object.
print("id if a is", id(a))

# Compare with a mutable object:
x = [1, 2]
print("id of x:", id(x))
x.append(3)  # This modifies the original list object in place.
print("id of x:", id(x)) # id remains the same


In [5]:
# There is sound logic in why variables where made immutable
# === 1. Thread Safety ===
# Immutable objects are inherently thread-safe because their values cannot change once created.
# This eliminates the risk of race conditions when multiple threads access the same object.

import threading

# Shared variable
shared_value = 42
lock = threading.Lock()  # Synchronization mechanism

def modify_value():
    global shared_value
    with lock:  # Acquire the lock before modifying shared_value
        # Read the current value of shared_value
        current_value = shared_value
        # Compute the new value
        new_value = current_value + 1
        # Reassign shared_value to the new value
        shared_value = new_value
        print(f"Thread {threading.current_thread().name}: shared_value = {shared_value}")

# Create two threads
thread1 = threading.Thread(target=modify_value, name="Thread-1")
thread2 = threading.Thread(target=modify_value, name="Thread-2")

thread1.start()
thread2.start()
thread1.join()
thread2.join()


# === 2. Hashability ===
# Immutable objects can be used as keys in dictionaries or elements in sets because their hash value remains constant.
# Mutable objects, like lists, cannot be hashed because their contents can change.

# Immutable object (can be used as a dictionary key)
immutable_key = "hello"
my_dict = {immutable_key: "world"}
print(f"Dictionary with immutable key: {my_dict}")  # Output: {'hello': 'world'}

# Mutable object (cannot be used as a dictionary key)
mutable_key = [1, 2, 3]
try:
    my_dict[mutable_key] = "error"
except TypeError as e:
    print(f"Error: {e}")  # Output: unhashable type: 'list'

# Example of hashability with tuples (immutable) vs. lists (mutable)
immutable_tuple = (1, 2, 3)
mutable_list = [1, 2, 3]

h1 = hash(immutable_tuple)  # Works fine (tuples are immutable)
#h2 = hash(mutable_list)  # Raises TypeError (lists are mutable)
print(str(h1))
try:
    hash(mutable_list)  # Raises TypeError
except TypeError as e:
    print(f"Error: {e}")  # Output: unhashable type: 'list'


# === 3. Predictability ===
# Immutable objects ensure that their value will not change unexpectedly, making programs easier to reason about.

# Immutable object (predictable behavior)
a = 5
b = a
a += 1  # Creates a new integer object; `b` remains unchanged
print(f"a = {a}, b = {b}")  # Output: a = 6, b = 5

# Mutable object (unpredictable behavior)
my_list = [1, 2, 3]
another_list = my_list
my_list.append(4)  # Modifies the original list; `another_list` reflects the change
print(f"my_list = {my_list}, another_list = {another_list}")  # Output: [1, 2, 3, 4], [1, 2, 3, 4]


# === 4. Memory Efficiency ===
# Immutability allows Python to reuse objects in memory through techniques like interning and object pooling.

# Interned integers (memory-efficient)
a = 42
b = 42
print(f"a is b: {a is b}")  # Output: True (same object in memory)

# Non-interned integers (separate objects)
c = 257
d = 257
print(f"c is d: {c is d}")  # Output: False (different objects in memory)


# === 5. Functional Programming ===
# Immutability aligns with functional programming principles, where functions avoid side effects and work with pure data.

# Pure function (no side effects)
def add_one(x):
    return x + 1

a = 5
b = add_one(a)
print(f"a = {a}, b = {b}")  # Output: a = 5, b = 6 (a remains unchanged)

# Impure function (side effects)
def append_to_list(lst, value):
    lst.append(value)

my_list = [1, 2, 3]
append_to_list(my_list, 4)
print(f"my_list = {my_list}")  # Output: [1, 2, 3, 4] (original list is modified)


# === 6. Simplified Semantics ===
# Immutability simplifies operations like copying and comparing objects.

# Immutable object (simple semantics)
a = "hello"
b = "hello"
print(f"a == b: {a == b}")  # Output: True (values are equal)
print(f"a is b: {a is b}")  # Output: True (same object in memory)

# Mutable object (complex semantics)
my_list = [1, 2, 3]
another_list = my_list[:]  # Shallow copy
print(f"my_list == another_list: {my_list == another_list}")  # Output: True (values are equal)
print(f"my_list is another_list: {my_list is another_list}")  # Output: False (different objects in memory)

Thread Thread-1: shared_value = 43
Thread Thread-2: shared_value = 44
Dictionary with immutable key: {'hello': 'world'}
Error: unhashable type: 'list'
529344067295497451
Error: unhashable type: 'list'
a = 6, b = 5
my_list = [1, 2, 3, 4], another_list = [1, 2, 3, 4]
a is b: True
c is d: False
a = 5, b = 6
my_list = [1, 2, 3, 4]
a == b: True
a is b: True
my_list == another_list: True
my_list is another_list: False


In [ ]:
import sys

s1 = sys.intern('long_string_example_that_repeats')
s2 = sys.intern('long_string_example_that_repeats')
print("s1 is s2:", s1 is s2)  # Interning ensures identity match

# Without interning
s3 = 'long_string_example_that_repeats'
s4 = 'long_string_example_that_repeats'
print("s3 is s4:", s3 is s4)  # May be True or False depending on interning behavior

s1 is s2: True
s3 is s4: True


## 3. Object Headers and Memory Layout
All Python objects carry metadata: reference count, type pointer, etc. The `sys.getsizeof()` function returns the size of the object in bytes (not counting referenced contents for containers).

In [ ]:
import sys

print("Size of int(5):", sys.getsizeof(5))
print("Size of list []:", sys.getsizeof([]))
print("Size of dict {}:", sys.getsizeof({}))
print("Size of 'a':", sys.getsizeof('a'))

Size of int(5): 28
Size of list []: 56
Size of dict {}: 64
Size of 'a': 50


## 4. Performance Implications
- Using `is` is faster than `==` for strings if interning is used.
- Interning is especially helpful in dictionary-heavy or loop-intensive code.
- Object creation has memory and GC overhead—reuse when possible.

In [ ]:
import timeit

print("Time using 'is':", timeit.timeit('"hello" is "hello"', number=1_000_000))
print("Time using '==':", timeit.timeit('"hello" == "hello"', number=1_000_000))

## Challenge: Optimize Dictionary Lookup with Interning
Given many repeated string keys in a loop-heavy dictionary lookup:
- Profile with and without `sys.intern()`.
- Observe performance and memory impact.

In [ ]:
import random
import string
import time
import sys

# Generate 10000 repeated keys
keys = ['user_' + random.choice(string.ascii_letters) for _ in range(10000)]
data = {sys.intern(k): i for i, k in enumerate(keys)}  # Intern keys

start = time.time()
for k in keys:
    _ = data[sys.intern(k)]  # Intern during lookup
print("Time with interning:", time.time() - start)

# Now try without interning
data2 = {k: i for i, k in enumerate(keys)}  # No interning

start = time.time()
for k in keys:
    _ = data2[k]
print("Time without interning:", time.time() - start)

### Summary
- Use `is` for identity, `==` for value.
- Intern small strings or repeated keys for performance.
- Know what `sys.getsizeof()` includes—and what it does not.
- Avoid excessive object creation in tight loops.

Up next: Exploring the Python bytecode and how it really runs your code.